## Importing Libraries

In [2]:
import cv2 as cv
import numpy as np
import os


### Add some configs

In [4]:
# Configuration
CHESS_BOARD_DIM = (7, 7)
image_dir_path = "images"
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# Create images directory if not exists
if not os.path.isdir(image_dir_path):
    os.makedirs(image_dir_path)
    print(f'"{image_dir_path}" Directory is created')
else:
    print(f'"{image_dir_path}" Directory already Exists.')



def detect_checker_board(image, grayImage, criteria, boardDimension):
    ret, corners = cv.findChessboardCorners(grayImage, boardDimension)
    if ret:
        corners1 = cv.cornerSubPix(grayImage, corners, (3, 3), (-1, -1), criteria)
        image = cv.drawChessboardCorners(image, boardDimension, corners1, ret)
    return image, ret

n = 0  # Image counter

"images" Directory already Exists.


### Capture checkboard images

In [9]:
ip_address = "192.168.1.9"
port = "8080"
video_url = f"http://{ip_address}:{port}/video"

# Capture and save checkerboard images
cap = cv.VideoCapture(video_url)


while True:
    ret, frame = cap.read()
    if not ret:
        break
    copyFrame = frame.copy()
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

    image, board_detected = detect_checker_board(frame, gray, criteria, CHESS_BOARD_DIM)

    cv.putText(frame, f"saved_img : {n}", (30, 40), cv.FONT_HERSHEY_PLAIN, 1.4, (0, 255, 0), 2, cv.LINE_AA)
    cv.imshow("frame", frame)
    cv.imshow("copyFrame", copyFrame)

    key = cv.waitKey(1)
    if key == ord("q"):
        break
    if key == ord("s") and board_detected:
        cv.imwrite(f"{image_dir_path}/image{n}.png", copyFrame)
        print(f"saved image number {n}")
        n += 1


cap.release()
cv.destroyAllWindows()
print("Total saved Images:", n)

saved image number 4
saved image number 5
saved image number 6
saved image number 7
saved image number 8
saved image number 9
saved image number 10
saved image number 11
saved image number 12
saved image number 13
saved image number 14
saved image number 15
saved image number 16
saved image number 17
saved image number 18
saved image number 19
saved image number 20
saved image number 21
saved image number 22
saved image number 23
saved image number 24
Total saved Images: 25


## Calibration

In [10]:
# -------------------------------------
# Calibration Phase
# -------------------------------------
print("\nStarting Calibration...")

n = len(os.listdir(image_dir_path))
# Prepare object points
objp = np.zeros((CHESS_BOARD_DIM[0]*CHESS_BOARD_DIM[1], 3), np.float32)
objp[:, :2] = np.mgrid[0:CHESS_BOARD_DIM[0], 0:CHESS_BOARD_DIM[1]].T.reshape(-1, 2)

objpoints = []  # 3D points in real world space
imgpoints = []  # 2D points in image plane

# Load saved images and detect corners
for img_name in os.listdir(image_dir_path):
    img = cv.imread(f"{image_dir_path}/{img_name}")
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

    ret, corners = cv.findChessboardCorners(gray, CHESS_BOARD_DIM, None)
    if ret:
        objpoints.append(objp)
        corners2 = cv.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
        imgpoints.append(corners2)

# Perform calibration
ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
print("Camera matrix:\n", mtx)
print("Distortion coefficients:\n", dist)

# -------------------------------------
# Undistort one image and crop it
# -------------------------------------
img = cv.imread(f"{image_dir_path}/image4.png")
h, w = img.shape[:2]
newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w, h), 1, (w, h))

# Undistort
mapx, mapy = cv.initUndistortRectifyMap(mtx, dist, None, newcameramtx, (w, h), 5)
dst = cv.remap(img, mapx, mapy, cv.INTER_LINEAR)

# Crop and save
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
cv.imwrite('calibresult.png', dst)
print("Undistorted and cropped image saved as 'calibresult.png'")

# -------------------------------------
# Re-projection error
# -------------------------------------
mean_error = 0
for i in range(len(objpoints)):
    imgpoints2, _ = cv.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
    error = cv.norm(imgpoints[i], imgpoints2, cv.NORM_L2) / len(imgpoints2)
    mean_error += error

print("Total re-projection error: {}".format(mean_error / len(objpoints)))



Starting Calibration...
Camera matrix:
 [[1.08431524e+03 0.00000000e+00 5.43015050e+02]
 [0.00000000e+00 1.10519712e+03 7.76066174e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
Distortion coefficients:
 [[ 0.09500513 -0.27888403  0.00167986 -0.00409425  0.20518896]]
Undistorted and cropped image saved as 'calibresult.png'
Total re-projection error: 0.0979019459009365


In [11]:
import cv2 as cv
import os
import numpy as np

# Checker board size
CHESS_BOARD_DIM = (7, 7)

# The size of Square in the checker board.
SQUARE_SIZE = 19  # millimeters

# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)


calib_data_path = "calib_data"
CHECK_DIR = os.path.isdir(calib_data_path)


if not CHECK_DIR:
    os.makedirs(calib_data_path)
    print(f'"{calib_data_path}" Directory is created')

else:
    print(f'"{calib_data_path}" Directory already Exists.')

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
obj_3D = np.zeros((CHESS_BOARD_DIM[0] * CHESS_BOARD_DIM[1], 3), np.float32)

obj_3D[:, :2] = np.mgrid[0 : CHESS_BOARD_DIM[0], 0 : CHESS_BOARD_DIM[1]].T.reshape(-1, 2)
obj_3D *= SQUARE_SIZE
print(obj_3D)

# Arrays to store object points and image points from all the images.
obj_points_3D = []  # 3d point in real world space
img_points_2D = []  # 2d points in image plane.

# The images directory path
image_dir_path = "images"

files = os.listdir(image_dir_path)
for file in files:
    print(file)
    imagePath = os.path.join(image_dir_path, file)
    # print(imagePath)

    image = cv.imread(imagePath)
    grayScale = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    ret, corners = cv.findChessboardCorners(image, CHESS_BOARD_DIM, None)
    if ret == True:
        obj_points_3D.append(obj_3D)
        corners2 = cv.cornerSubPix(grayScale, corners, (3, 3), (-1, -1), criteria)
        img_points_2D.append(corners2)

        img = cv.drawChessboardCorners(image, CHESS_BOARD_DIM, corners2, ret)

cv.destroyAllWindows()
# h, w = image.shape[:2]
ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(
    obj_points_3D, img_points_2D, grayScale.shape[::-1], None, None)
print("calibrated")

print("duming the data into one files using numpy ")
np.savez(
    f"{calib_data_path}/MultiMatrix",
    camMatrix=mtx,
    distCoef=dist,
    rVector=rvecs,
    tVector=tvecs,
)

"calib_data" Directory is created
[[  0.   0.   0.]
 [ 19.   0.   0.]
 [ 38.   0.   0.]
 [ 57.   0.   0.]
 [ 76.   0.   0.]
 [ 95.   0.   0.]
 [114.   0.   0.]
 [  0.  19.   0.]
 [ 19.  19.   0.]
 [ 38.  19.   0.]
 [ 57.  19.   0.]
 [ 76.  19.   0.]
 [ 95.  19.   0.]
 [114.  19.   0.]
 [  0.  38.   0.]
 [ 19.  38.   0.]
 [ 38.  38.   0.]
 [ 57.  38.   0.]
 [ 76.  38.   0.]
 [ 95.  38.   0.]
 [114.  38.   0.]
 [  0.  57.   0.]
 [ 19.  57.   0.]
 [ 38.  57.   0.]
 [ 57.  57.   0.]
 [ 76.  57.   0.]
 [ 95.  57.   0.]
 [114.  57.   0.]
 [  0.  76.   0.]
 [ 19.  76.   0.]
 [ 38.  76.   0.]
 [ 57.  76.   0.]
 [ 76.  76.   0.]
 [ 95.  76.   0.]
 [114.  76.   0.]
 [  0.  95.   0.]
 [ 19.  95.   0.]
 [ 38.  95.   0.]
 [ 57.  95.   0.]
 [ 76.  95.   0.]
 [ 95.  95.   0.]
 [114.  95.   0.]
 [  0. 114.   0.]
 [ 19. 114.   0.]
 [ 38. 114.   0.]
 [ 57. 114.   0.]
 [ 76. 114.   0.]
 [ 95. 114.   0.]
 [114. 114.   0.]]
calibresult.png
image10.png
image11.png
image16.png
image17.png
image18.png
image19

## Loading camera parameters

In [12]:
print("loading data stored using numpy savez function\n \n \n")

calib_data_path = "calib_data"
CHECK_DIR = os.path.isdir(calib_data_path)

data = np.load(f"{calib_data_path}/MultiMatrix.npz")

camMatrix = data["camMatrix"]
distCof = data["distCoef"]
rVector = data["rVector"]
tVector = data["tVector"]

print("loaded calibration data successfully")

print("Camera Matrix (camMatrix):\n", camMatrix)
print("\nDistortion Coefficients (distCof):\n", distCof)
print("\nRotation Vectors (rVector):\n", rVector)
print("\nTranslation Vectors (tVector):\n", tVector)

loading data stored using numpy savez function
 
 

loaded calibration data successfully
Camera Matrix (camMatrix):
 [[1.18516243e+03 0.00000000e+00 5.37934227e+02]
 [0.00000000e+00 1.21709038e+03 7.84347318e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]

Distortion Coefficients (distCof):
 [[ 0.05643447 -0.22052398  0.00293713 -0.00673012  0.20358823]]

Rotation Vectors (rVector):
 [[[ 0.22437973]
  [-0.03920785]
  [ 0.02024531]]

 [[ 0.33231046]
  [-0.01745332]
  [ 0.03571354]]

 [[ 0.26602658]
  [-0.07612129]
  [ 0.07264228]]

 [[ 0.18409498]
  [-0.04084824]
  [ 0.09553331]]

 [[ 0.23723277]
  [-0.01742499]
  [ 0.06810447]]

 [[ 0.25303438]
  [-0.01293337]
  [ 0.04208241]]

 [[ 0.21012122]
  [-0.04311699]
  [ 0.07367884]]

 [[ 0.27358896]
  [ 0.04932087]
  [ 0.12432099]]

 [[ 0.26224797]
  [ 0.00727671]
  [ 0.03839566]]

 [[ 0.28441074]
  [-0.01047044]
  [ 0.01699965]]

 [[ 0.26707627]
  [-0.01988055]
  [ 0.03853436]]

 [[ 0.28154867]
  [-0.00881848]
  [ 0.05257864]]

 [[ 0.2